In [51]:
import pandas as pd
import numpy as np
import datetime as dt
from scipy import stats

In [52]:
df = pd.read_csv('/home/roman/git_projects/RFM_analysis/bonuscheques_202312252155.csv')
df.head()

,datetime,shop,card,bonus_earned,bonus_spent,summ,summ_with_disc,doc_id
0,2021-07-13 12:56:09.000,Аптека 2,2000200195023,51,0,3400,3400,15#2002741#65938#2_29
1,2021-07-30 10:42:00.000,Аптека 2,2000200193494,57,0,747,747,15#2002972#65955#2_5
2,2021-10-11 12:55:23.000,Аптека 2,2000200199106,92,253,3077,3077,15#2004060#66028#2_29
3,2021-10-14 14:48:56.000,Аптека 2,2000200168768,1,0,54,54,15#2004107#66031#2_57
4,2021-10-20 11:09:39.000,Аптека 2,2000200226314,101,0,1733,1733,15#2004192#66037#2_16


In [53]:
df.shape

(38486, 8)

In [54]:
df.dtypes

datetime          object
shop              object
card              object
bonus_earned       int64
bonus_spent        int64
summ               int64
summ_with_disc     int64
doc_id            object
dtype: object

In [55]:
df.isna().sum()

datetime          0
shop              0
card              0
bonus_earned      0
bonus_spent       0
summ              0
summ_with_disc    0
doc_id            0
dtype: int64

In [56]:
df['datetime'] = df['datetime'].apply(pd.to_datetime)
snapshot_date = df['datetime'].max()
df_RFM = df.groupby('card').agg({
    'datetime': lambda x: (snapshot_date - x.max()).days,
    'card': 'count',
    'summ_with_disc': 'sum'
})

In [57]:
df_RFM

,datetime,card,summ_with_disc
card,,,
001848d0-e56a-412e-be0a-423dfa0e564f,26,7,4391
001cb0a0-e62e-47a9-995e-599bdc7f8750,27,12,5887
00572591-c154-4b9f-83e4-3ce5b00bf9c8,244,1,557
00625996-f49f-4863-8132-40d08e99b2da,56,4,4755
00703805-f16d-4b83-b7ad-affb2736aa0d,15,1,1555
...,...,...,...
ff9695c5-6e8c-4446-adc6-944b2731b8f7,172,1,559
ffbf9d44-679e-412a-9927-ac6ce80c49a2,286,1,539
ffd22196-662c-4266-b3be-024a42cad78b,4,3,2052


In [58]:
df_RFM.rename(columns={
    'datetime': 'Recency',
    'card': 'Frequency',
    'summ_with_disc': 'Monetary'
}, inplace=True)

In [59]:
df_RFM

,Recency,Frequency,Monetary
card,,,
001848d0-e56a-412e-be0a-423dfa0e564f,26,7,4391
001cb0a0-e62e-47a9-995e-599bdc7f8750,27,12,5887
00572591-c154-4b9f-83e4-3ce5b00bf9c8,244,1,557
00625996-f49f-4863-8132-40d08e99b2da,56,4,4755
00703805-f16d-4b83-b7ad-affb2736aa0d,15,1,1555
...,...,...,...
ff9695c5-6e8c-4446-adc6-944b2731b8f7,172,1,559
ffbf9d44-679e-412a-9927-ac6ce80c49a2,286,1,539
ffd22196-662c-4266-b3be-024a42cad78b,4,3,2052


In [60]:
df_RFM.dtypes

Recency      int64
Frequency    int64
Monetary     int64
dtype: object

In [61]:
df_RFM[['Recency', 'Frequency', 'Monetary']].describe()

,Recency,Frequency,Monetary
count,9394.000000,9394.00000,9394.000000
mean,112.924420,4.09687,3416.820098
std,95.124941,6.43155,6388.046844
min,0.000000,1.00000,21.000000
25%,29.000000,1.00000,731.250000
50%,87.000000,2.00000,1586.000000
75%,182.000000,5.00000,3731.750000
max,332.000000,217.00000,162687.000000


In [62]:
df_RFM[['Recency', 'Frequency', 'Monetary']].mode()

,Recency,Frequency,Monetary
0,2,1,619


In [63]:
pd.concat([df_RFM.eq(x) for _, x in df_RFM.mode().iterrows()]).sum()

Recency       133
Frequency    3784
Monetary       11
dtype: int64

In [64]:
df_RFM.query('Recency in (0, 1)')['Recency'].count()

248

In [65]:
df_RFM.query('Frequency == 1')['Monetary'].sort_values(ascending=False).head(15)

card
e77d379f-87c9-4986-b2f1-0f841a3d9ade    10869
2000200211426                           10531
2000200249696                           10102
1e2c4501-2da6-4e42-9fa7-51b0a4d1cec1     9801
2000200194224                            9793
2000200208877                            9092
2000200241225                            8668
2000200216056                            8557
2000200241430                            8495
2000200221425                            8395
2000200194316                            7993
2000200238409                            7928
2000200248019                            7920
2000200227595                            7404
2000200209669                            7028
Name: Monetary, dtype: int64

In [66]:
round((df_RFM.query('Frequency == 1')['Monetary'].sum() / df_RFM['Monetary'].sum()) * 100)

12

- около 12 процентов составляет сумма, принесенная клиентами, которые сделали лишь один заказ. 12 процентов относительно всей суммы цифра немалая, но учитывая, что ее принесли более одной трети клиентов, а значит остальные деньги принесли другие две-трети клиентов, тех, кто был лишь один раз и сделал лишь один заказ при построении итогового RFM-запроса мы отсеивать не будем - любой клиент, даже сделавший один заказ, в будущем это потенциальный постоянный клиент. Так как есть разбросы в медианах и средних значениях, посмотрим и сравним данные значения с усеченными данными, уберем по 5% от минимума и максимума:

In [67]:
stats.trim_mean(df_RFM, 0.05)

array([ 107.94311731,    3.1711211 , 2514.76300851])

In [68]:
df_RFM['new_column'] = np.where(df_RFM['Recency']<100, '< 100 days',
                   np.where(df_RFM['Recency']<200, '>= 100 and < 200 days',
                   np.where(df_RFM['Recency']<300, '>= 200 and < 300 days', '>= 300 days')))

In [72]:
df_RFM = df_RFM.reset_index()
df_RFM

,card,Recency,Frequency,Monetary,new_column
0,001848d0-e56a-412e-be0a-423dfa0e564f,26,7,4391,< 100 days
1,001cb0a0-e62e-47a9-995e-599bdc7f8750,27,12,5887,< 100 days
2,00572591-c154-4b9f-83e4-3ce5b00bf9c8,244,1,557,>= 200 and < 300 days
3,00625996-f49f-4863-8132-40d08e99b2da,56,4,4755,< 100 days
4,00703805-f16d-4b83-b7ad-affb2736aa0d,15,1,1555,< 100 days
...,...,...,...,...,...
9389,ff9695c5-6e8c-4446-adc6-944b2731b8f7,172,1,559,>= 100 and < 200 days
9390,ffbf9d44-679e-412a-9927-ac6ce80c49a2,286,1,539,>= 200 and < 300 days
9391,ffd22196-662c-4266-b3be-024a42cad78b,4,3,2052,< 100 days
9392,ffd93478-ad6d-494f-a6ab-f7f0c8058488,99,7,2294,< 100 days


In [73]:
df_RFM.groupby('new_column')['card'].count()

new_column
< 100 days               5100
>= 100 and < 200 days    2274
>= 200 and < 300 days    1588
>= 300 days               432
Name: card, dtype: int64

Результат - 5100, 2274, 1588 и 432 (если сложить все четыре итога, то получится 9394, количество уникальных клиентов, значит, потерь нет). 5100 клиентов посетили аптеку за последние сто дней, это более половины из всех.